In [1]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))

import shutil
import datetime
import numpy as np
import pandas as pd

from library import common as cm
from library import regression_aux as raux

In [2]:
%run setup.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.random.seed(RANDOM_SEED)

In [3]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

We remove samples when the next trade is not available. 163472 samples (4.52%) are removed. We have 95.48% of original data left, yielding a size of 3453492.
We remove unreasonable implvol0. 3159 samples (0.09%) are removed. We have 95.39% of original data left, yielding a size of 3450333.
We remove samples that have time to maturity less than 0 day. 0 samples (0.00%) are removed. We have 95.39% of original data left, yielding a size of 3450333.
We remove in-the-money samples. 712998 samples (20.66%) are removed. We have 75.68% of original data left, yielding a size of 2737335.
We shrink moneyness range. 104752 samples (3.83%) are removed. We have 72.78% of original data left, yielding a size of 2632583.


In [4]:
print(f'The number of samples after loading is : {df.shape[0]}')

regr_dir = res_dir + 'Regression/'
print(f'Results are saved at:\n{regr_dir}')

The number of samples after loading is : 2632583
Results are saved at:
C:\Users\Weiguan\Dropbox\Research\01_DeepHedging\Data_OK\OptionMetrics\Result/FREQ=1D_HALFMONEY=otm_MINM=0.8_MAXM=1.5_MINTAU=0_Permute=False_VIX=True_MISSING=replace_missing_WINDOW=180D_KickTmrZeroVolume=False/Regression/


In [5]:
if os.path.exists(regr_dir):
    shutil.rmtree(regr_dir)
os.makedirs(regr_dir)
shutil.copy('setup.py', regr_dir);

### NO permute

In [6]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')

#### No hedge

In [7]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}No_Hedge/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    zero = np.array([0.]*len(df))
    zero = pd.Series(zero, index=df.index)

    cm.store_pnl(df, zero, pnl_path)
    save_tables(None, sub_regr_dir)

#### Baseline: Black-Scholes 

In [8]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}BS_Benchmark/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    cm.store_pnl(df, df['delta_bs'], pnl_path)
    save_tables(None, sub_regr_dir)

#### Delta-Vega-Vanna

In [9]:
if not PERMUTE:
    sub_regr_dir = f'{regr_dir}Delta_Vega_Vanna/'
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vega_n', 'vanna_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }

    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)

VIX is used!


### Permute

#### No Hedge

In [10]:
if PERMUTE:
    df_permute = df.copy()
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        sub_res = f'{regr_dir}No_Hedge/pnl/'
        os.makedirs(sub_res, exist_ok=True)
        pnl_path = f'{sub_res}pnl{i}.csv'
        zero = np.array([0.]*len(df_permute))
        zero = pd.Series(zero, index=df_permute.index)
        cm.store_pnl(df_permute, zero, pnl_path)
    del df_permute

#### BS 

In [11]:
if PERMUTE:
    df_permute = df.copy()
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        sub_res = f'{regr_dir}BS_Benchmark/pnl/'
        os.makedirs(sub_res, exist_ok=True)
        pnl_path = f'{sub_res}pnl{i}.csv'
        cm.store_pnl(df_permute, df_permute['delta_bs'], pnl_path)
    del df_permute

#### Delta-Vega-Vanna

In [12]:
if PERMUTE:
    df_permute = df.copy()
    sub_regr_dir = f'{regr_dir}Delta_Vega_Vanna/'
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        kwargs = {'vix': VIX, 
              'features': ['delta_bs', 'vega_n', 'vanna_n'], 
              'max_period': max_period, 
              'sub_res': f'{sub_regr_dir}pnl/',
              'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
              'df': df_permute,
             'delta_coeff_1': False,
                 'agg_side': False}
        
        res = raux.run_store_lin(**kwargs)